In [1]:
from pvutils import pvutils as pvu
import math
import datetime
import pytz
from datetime import datetime
import numpy as np
import pandas as pd
# ejemplo: America/Vancouver'

In [2]:
# Parametro de ejemplo_:

# Buenos Aires
ls = 60    # longitud meridiana
le= 58.48  # 58°29' W


# fecha1 = pd.date_range('2023-10-16 12:00:00', '2023-10-16 12:00:00')

fecha1 = pd.to_datetime('2023-10-16 10:00:00', format='%Y-%m-%d %H:%M:%S')


fecha1



Timestamp('2023-10-16 10:00:00')

In [3]:
# Día juviano_:

# for fecha0 in fecha1:
#     print(pvu.julian_day(fecha0,TZ='America/Vancouver')) # dia del año


Dj = pvu.julian_day(fecha1,TZ='America/Argentina/Buenos_Aires') # dia del año

"""
nota:

El dia juliano no es necesarion el TZ. creo,. 

"""



'\nnota:\n\nEl dia juliano no es necesarion el TZ. creo,. \n\n'

In [4]:
# Ángulo diario_:

# for fecha0 in fecha1:
#     print(pvu.angulo_diario(fecha0,unit='rad'))

Adia = pvu.angulo_diario(fecha1,unit='rad')

In [5]:
# Ecuacion del tiempo_:

# for fecha0 in fecha1:
#     print(pvu.ecc_del_tiempo(pvu.angulo_diario(fecha0,unit='rad')))

Et = pvu.ecc_del_tiempo(Adia,unit='rad')


## Tiempo solar verdadero

In [6]:
# Tiempo Solar Verdadero_:

# Tsolar = pvu.tiempo_solar_verdadero(hora_local_std = hra, utc= 'America/Vancouver', ecc_del_tiempo= Et, long_meridian= ls, long_ubicacion= le )

Tsolar = pvu.tsv(hora_local_std = fecha1, utc= 'America/Argentina/Buenos_Aires', ecc_del_tiempo= 14.62, long_meridian= ls, long_ubicacion= le )

Tsolar

datetime.datetime(2023, 10, 16, 10, 14, 37, tzinfo=<DstTzInfo 'America/Argentina/Buenos_Aires' LMT-1 day, 20:06:00 STD>)

In [ ]:
#Tiempo Solar Verdadero:

# Tsolar = pvu.tiempo_solar_verdadero(hora_local_std = hra, utc= 'America/Vancouver', ecc_del_tiempo= Et, long_meridian= ls, long_ubicacion= le )

hra = datetime(2023, 1, 1, 12, 0, 0)

Tsolar = pvu.tsv(hora_local_std = hra, utc= 'America/Argentina/Buenos_Aires', ecc_del_tiempo= Et, long_meridian= ls, long_ubicacion= le )

# Imprimir en formato string DD/MM/YYYY HH:MM:SS
Tsolar.strftime('%d/%m/%Y %H:%M:%S')

In [ ]:
def tsv2(hora_local_std: datetime, utc: str, ecc_del_tiempo: float, long_meridian: float, long_ubicacion: float) -> float:
    """
    Calcula el tiempo solar verdadero (TSV) para una hora local dada.

    Args:
        hora_local: hora local (datetime)
        utc: zona horaria, valor por defecto 'UTC'
        ecc_del_tiempo: ecuación del tiempo (minutos)
        long_meridian: longitud del meridiano (grados o radianes)
        long_ubicacion: longitud de la ubicación (grados o radianes)
        unit: unidades del ángulo ('deg' para grados o 'rad' para radianes)
    Returns:
        tiempo_solar_verdadero: tiempo solar verdadero (datetime)
    """
    hora_local_std = hora_local_std.replace(tzinfo=pytz.timezone(utc))
    # compute the TSV in minutes
    tsv = hora_local_std.hour*60 + hora_local_std.minute + hora_local_std.second/60 + ecc_del_tiempo + 4*(long_meridian-long_ubicacion)/24/60
    # convert to datetime
    tsv = datetime.datetime(hora_local_std.year, hora_local_std.month, hora_local_std.day, int(tsv/60), int(tsv%60), int((tsv%1)*60), tzinfo=pytz.timezone(utc))

    return tsv2



In [11]:
fecha1 = datetime(2023, 1, 1, 10, 0, 0)

In [17]:
utc= 'America/Argentina/Buenos_Aires'
hora_local_std = fecha1
ecc_del_tiempo= Et
long_meridian= ls
long_ubicacion= le

hora_local_std = hora_local_std.replace(tzinfo=pytz.timezone(utc))
# compute the TSV in minutes
tsv = hora_local_std.hour*60 + hora_local_std.minute + hora_local_std.second/60 + ecc_del_tiempo + 4*(long_meridian-long_ubicacion)
# convert to datetime
tsv = datetime(hora_local_std.year, hora_local_std.month, hora_local_std.day, int(tsv/60), int(tsv%60), int((tsv%1)*60), tzinfo=pytz.timezone(utc))


# Tsolar = tsv2(hora_local_std = fecha1, utc= 'America/Argentina/Buenos_Aires', ecc_del_tiempo= Et, long_meridian= ls, long_ubicacion= le )

# Tsolar
tsv

datetime.datetime(2023, 1, 1, 10, 20, 42, tzinfo=<DstTzInfo 'America/Argentina/Buenos_Aires' LMT-1 day, 20:06:00 STD>)

In [9]:
# 4(Ls -le) + Et

""""
la parte entera son los minutos = 20 

la decimal se obtiene con la regla 3 simple:   1 = 60 seg
"""

4*(60-58.48) + 14.62

20.70000000000001

## Continuación de la validación

In [12]:
# Angulo horario_:

Tsolar = hra = datetime(2023, 10, 16, 10, 20, 41)  # creación de un TSV para la validacion:

Ahrs = pvu.angulo_horario(tsv= Tsolar, unit='rad')

In [10]:
# Declinación Solar_:

DecSol  = pvu.declinacion_solar(angulo_diario = Adia)


In [16]:
# cos_Sza_:

CosZA = pvu.cos_sza(latitude= le, hour_angle=Ahrs , solar_declination= DecSol)


In [20]:
# ángulo cenital_:

SZA = pvu.sza(CosZA, unit='rad')

In [23]:
# Distancia Tierra sol_:

Tir_Sol = pvu.distancia_tierra_sol(angulo_diario = Adia)

In [24]:
# Irradiancia Solar_: 

Io = pvu.irradiancia_extra_sup_horizontal(irradiacion_extraterrestre = 1367,distancia_tierra_sol= Tir_Sol, cos_zsa = CosZA)


"""
nota:

la iiradiancia extraterrestes es constante, por tanto, peude ser un numero fijo en "1367" 

"""


'\nnota:\n\nla iiradiancia extraterrestes es constante, por tanto, peude ser un numero fijo en "1367" \n\n'